In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
import lightgbm as lgb
import pickle

In [50]:
train_df = pd.read_csv('./Train.csv')
test_df = pd.read_csv('./Test.csv')

In [15]:
if 'Place_ID' in train_df.columns and 'Date' in train_df.columns:
    train_df = train_df.drop(['Place_ID', 'Date'], axis=1)

In [16]:
if 'Place_ID X Date' in train_df.columns:
    train_df[['Place_ID', 'Date']] = train_df['Place_ID X Date'].str.split(' X ', expand=True)
    idx = train_df.columns.get_loc('Place_ID X Date')
    train_df = train_df.drop('Place_ID X Date', axis=1)
    train_df.insert(idx, 'Date', train_df.pop('Date'))
    train_df.insert(idx, 'Place_ID', train_df.pop('Place_ID'))
    train_df['Date'] = pd.to_datetime(train_df['Date'])
    train_df['Date'] = train_df['Date'].dt.strftime('%m/%d/%y')

In [17]:
place_groups = train_df.groupby('Place_ID')
numeric_cols = train_df.select_dtypes(include=np.number).columns
train_df[numeric_cols] = place_groups[numeric_cols].transform(lambda x: x.fillna(x.median()))
train_df = train_df.fillna(method='ffill')

In [18]:
train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df['year'] = train_df['Date'].dt.year
train_df['month'] = train_df['Date'].dt.month
train_df['week'] = train_df['Date'].dt.week
train_df['day'] = train_df['Date'].dt.day

C:\Users\user\AppData\Local\Temp\ipykernel_9636\4629417.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  train_df['week'] = train_df['Date'].dt.week


In [19]:
train_df['total_days_month'] = train_df['month'].apply(lambda x: 31 if x==1 else (28+31 if x==2 else (28+31+31 if x==1 else 28+30+31+31))) 
train_df['total_days'] = train_df['total_days_month'] + train_df['day']

In [20]:
X_train = train_df.drop(['Place_ID', 'Date', 'target', 'target_min', 'target_max', 'target_variance', 'target_count'], axis=1)
y_train = train_df['target']

In [61]:
X_trains, X_tests, y_trains, y_tests = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [99]:
params = {
    'objective' :'regression',
    'learning_rate' : 0.03,
    'num_iterations': 50000,
    'max_bins': 50, 
    'max_depth' :7 ,
    'num_leaves' : 70,
    'feature_fraction': 0.64, 
    'bagging_fraction': 0.8, 
    'bagging_freq':1,
    'boosting_type' : 'gbdt',
    'metric': 'rmse' ,
     'min_data_in_leaf':5,
    'reg_lambda' :100
}



In [100]:
# lgb_trains = lgb.Dataset(X_trains, y_trains, silent=False,categorical_feature=['year','month','day','week'])
# lgb_evals = lgb.Dataset(X_tests, y_tests, silent=False,categorical_feature=['year','month','day','week'])
lgb_trains = lgb.Dataset(X_train, y_train, silent=False,categorical_feature=['year','month','day','week'])
lgb_evals = lgb.Dataset(X_train, y_train, silent=False,categorical_feature=['year','month','day','week'])
model = lgb.train(params, train_set = lgb_trains, num_boost_round=10000,verbose_eval=500, valid_sets=[lgb_trains,lgb_evals])


c:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\user\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\user\anaconda3\lib\site-packages\lightgbm\basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017053 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3806
[LightGBM] [Info] Number of data points in the train set: 30557, number of used features: 79
[LightGBM] [Info] Start training from score 61.148045


c:\Users\user\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\Users\user\anaconda3\lib\site-packages\lightgbm\basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\user\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[500]	training's rmse: 18.4875	valid_1's rmse: 18.4875
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1000]	training's rmse: 13.5458	valid_1's rmse: 13.5458
[1500]	training's rmse: 10.409	valid_1's rmse: 10.409
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2000]	training's rmse: 8.19657	valid_1's rmse: 8.19657
[2500]	training's rmse: 6.57529	valid_1's rmse: 6.57529
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3000]	training's rmse: 5.34267	valid_1's rmse: 5.34267
[LightGBM] [Warning] No further splits with positive gain, best gai

In [101]:
place_groups = test_df.groupby('Place_ID')
numeric_cols = test_df.select_dtypes(include=np.number).columns
test_df[numeric_cols] = place_groups[numeric_cols].transform(lambda x: x.fillna(x.median()))
test_df = test_df.fillna(method='ffill')

In [102]:
if 'Place_ID' in test_df.columns and 'Date' in test_df.columns:
    test_df = test_df.drop(['Place_ID', 'Date'], axis=1)
if 'Place_ID X Date' in test_df.columns:
    test_df[['Place_ID', 'Date']] = test_df['Place_ID X Date'].str.split(' X ', expand=True)
    idx = test_df.columns.get_loc('Place_ID X Date')
    test_df.insert(idx, 'Date', test_df.pop('Date'))
    test_df.insert(idx, 'Place_ID', test_df.pop('Place_ID'))
    test_df['Date'] = pd.to_datetime(test_df['Date'])
    test_df['Date'] = test_df['Date'].dt.strftime('%m/%d/%y')

In [103]:
test_df['Date'] = pd.to_datetime(test_df['Date'])
test_df['year'] = test_df['Date'].dt.year
test_df['month'] = test_df['Date'].dt.month
test_df['week'] = test_df['Date'].dt.week
test_df['day'] = test_df['Date'].dt.day

C:\Users\user\AppData\Local\Temp\ipykernel_9636\655527865.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  test_df['week'] = test_df['Date'].dt.week


In [104]:
test_df['total_days_month'] = test_df['month'].apply(lambda x: 31 if x==1 else (28+31 if x==2 else (28+31+31 if x==1 else 28+30+31+31))) 
test_df['total_days'] = test_df['total_days_month'] + test_df['day']

In [105]:
X_test = test_df.drop(['Place_ID X Date','Place_ID', 'Date'], axis=1)

In [106]:
test_pred = model.predict(X_test)

In [107]:
output_df = pd.DataFrame({
    'Place_ID X Date': test_df['Place_ID X Date'],
    'target': test_pred
})
output_df.to_csv('predictions.csv', index=False)